# PyTestLab – Notebook GUI Demo

This short notebook shows how you can build **interactive controls** for *any* PyTestLab instrument **without writing specialized widget classes** – everything is declared in a few lines.

*The demo runs entirely in simulation mode – no hardware required.*

In [1]:
#Install ipywidgets if it's missing.  (Google Colab already has it; classic Jupyter may need it once.)
import subprocess, sys, importlib.util, warnings
if importlib.util.find_spec("ipywidgets") is None:
    warnings.warn("Installing ipywidgets…")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ipywidgets>=8"])

# Optional – nice layout in classic Notebook
try:
    import ipywidgets as w; w.install_nbextension(enable=True)
except Exception:
    pass

In [3]:
## 1. Create (simulated) instruments"""
import pytestlab

# We're inside Jupyter -> an event-loop is already running → simple helper

# psu = await pytestlab.instruments.AutoInstrument.from_config(
#     "keysight/EDU36311A", simulate=True
# )
# await psu.connect_backend()

# dmm = await pytestlab.instruments.AutoInstrument.from_config(
#     "keysight/EDU34450A", simulate=True
# )

awg = await pytestlab.instruments.AutoInstrument.from_config(
    "keysight/EDU33212A", simulate=True
)

await awg.connect_backend()

print("Simulated PSU & DMM ready → build GUI panels below…")

TypeError: SimBackendV2.__init__() got an unexpected keyword argument 'profile'

In [ ]:
""## 2. Build a PSU control panel (Channel 1)"""
from pytestlab.gui.builder import InstrumentPanel, Slider, Toggle, Button
psu = None
psu_panel = InstrumentPanel(
    title="PSU – CH1 (Sim)",
    inst=None,
    controls=[
        Slider(
            label="Voltage [V]",
            getter=lambda i: i.set_amplitude(1),
            setter=lambda i, v: i.set_voltage(1, v),
            min=0.0, max=6.0, step=0.05,
        ),
        Slider(
            label="Current [A]",
            getter=lambda i: i.read_current(1),
            setter=lambda i, c: i.set_current(1, c),
            min=0.0, max=5.0, step=0.05,
        ),
        Toggle(
            label="Output",
            getter=lambda i: i.output(1),
            setter=lambda i, state: i.output(1, state),
            on_desc="ON", off_desc="OFF",
        ),
        Button(
            label="⟳ Refresh read-back",
            action=lambda i, p: p.refresh(),
        ),
    ],
)

# Move the slider, toggle the output – the PSU simulation follows immediately.

In [ ]:
""## 3. Build a DMM one-click read-out"""
from pytestlab.config.multimeter_config import DMMFunction

dmm_panel = InstrumentPanel(
    title="DMM – DC Voltage (Sim)",
    inst=dmm,
    controls=[
        Button(
            label="Measure",
            action=lambda dev, p: dev.measure(DMMFunction.VOLT_DC).then(lambda m: print("Measured:", m)),
            style="success",
        )
    ],
)

# Click **Measure** – the value is printed and the coroutine runs in the background.

## Where to go next?

* Add more `Slider`, `Toggle` or `Button` controls – everything is declarative.
* Extend `pytestlab.gui.builder` with your own widget types (dropdowns, plots…).
* Replace `simulate=True` with real VISA addresses and you have a *live* lab dashboard!